In [1]:
import duckdb

In [2]:
con = duckdb.connect()
con.execute("INSTALL httpfs; LOAD httpfs;")
con.execute("INSTALL ducklake; LOAD ducklake;")

In [3]:
minioAuth = {
    'accessKey': 'minioadmin',
    'secretKey': 'miniopassword',
    'bucket': 'marketing-and-ecommerce',
    'endpoint': 'localhost:9000'
}

con.sql(f"SET s3_endpoint = '{minioAuth['endpoint']}';")
con.sql(f"SET s3_access_key_id = '{minioAuth['accessKey']}';")
con.sql(f"SET s3_secret_access_key = '{minioAuth['secretKey']}';")
con.sql("SET s3_use_ssl = false;") # Wajib untuk http (tanpa ssl)
con.sql("SET s3_url_style = 'path';") # Wajib untuk MinIO

In [4]:
catalogMetadata = r'B:\GitHub Repository\Dapodik-ELT-DataPipeline\dataLakehouse\catalog.ducklake'

In [5]:
con.sql(f"ATTACH 'ducklake:{catalogMetadata}' AS dataLakehouse;")
con.sql("USE dataLakehouse;")

IOException: IO Error: Failed to attach DuckLake MetaData "__ducklake_metadata_dataLakehouse" at path + "B:\GitHub Repository\Dapodik-ELT-DataPipeline\dataLakehouse\catalog.ducklake"Cannot open file "B:\GitHub Repository\Dapodik-ELT-DataPipeline\dataLakehouse\catalog.ducklake": The system cannot find the path specified.


In [ ]:
con.sql("SELECT * FROM ducklake_snapshots('dataLakehouse')")

┌─────────────┬───────────────────────────────┬────────────────┬───────────────────────────────────────────────────────────────┬─────────┬────────────────┬───────────────────┐
│ snapshot_id │         snapshot_time         │ schema_version │                            changes                            │ author  │ commit_message │ commit_extra_info │
│    int64    │   timestamp with time zone    │     int64      │                    map(varchar, varchar[])                    │ varchar │    varchar     │      varchar      │
├─────────────┼───────────────────────────────┼────────────────┼───────────────────────────────────────────────────────────────┼─────────┼────────────────┼───────────────────┤
│           0 │ 2025-12-11 16:54:27.968607+07 │              0 │ {schemas_created=[main]}                                      │ NULL    │ NULL           │ NULL              │
│           1 │ 2025-12-11 16:54:28.080394+07 │              1 │ {schemas_created=[raw]}                                

In [ ]:
con.sql("SELECT * FROM information_schema.schemata;")

┌───────────────────────────────────┬────────────────────┬──────────────┬───────────────────────────────┬──────────────────────────────┬────────────────────────────┬──────────┐
│           catalog_name            │    schema_name     │ schema_owner │ default_character_set_catalog │ default_character_set_schema │ default_character_set_name │ sql_path │
│              varchar              │      varchar       │   varchar    │            varchar            │           varchar            │          varchar           │ varchar  │
├───────────────────────────────────┼────────────────────┼──────────────┼───────────────────────────────┼──────────────────────────────┼────────────────────────────┼──────────┤
│ __ducklake_metadata_dataLakehouse │ main               │ duckdb       │ NULL                          │ NULL                         │ NULL                       │ NULL     │
│ dataLakehouse                     │ main               │ duckdb       │ NULL                          │ NULL     

In [ ]:
con.sql("SELECT * FROM raw.events;")

┌──────────┬─────────────────────┬─────────────┬────────────┬─────────────┬────────────┬─────────────┬────────────────┬─────────────┬───────────────┬──────────────────────┬──────────────────┐
│ event_id │      timestamp      │ customer_id │ session_id │ event_type  │ product_id │ device_type │ traffic_source │ campaign_id │ page_category │ session_duration_sec │ experiment_group │
│  int64   │      timestamp      │    int64    │   int64    │   varchar   │   double   │   varchar   │    varchar     │    int64    │    varchar    │        double        │     varchar      │
├──────────┼─────────────────────┼─────────────┼────────────┼─────────────┼────────────┼─────────────┼────────────────┼─────────────┼───────────────┼──────────────────────┼──────────────────┤
│   180658 │ 2021-03-14 13:48:38 │       34340 │     531998 │ view        │     1352.0 │ mobile      │ Organic        │           0 │ Home          │                 90.3 │ Control          │
│   180659 │ 2021-01-08 12:36:45 │      

In [ ]:
con.sql("SELECT * FROM raw.customers;")

┌─────────────┬─────────────┬─────────┬───────┬─────────┬──────────────┬─────────────────────┐
│ customer_id │ signup_date │ country │  age  │ gender  │ loyalty_tier │ acquisition_channel │
│    int64    │    date     │ varchar │ int64 │ varchar │   varchar    │       varchar       │
├─────────────┼─────────────┼─────────┼───────┼─────────┼──────────────┼─────────────────────┤
│           1 │ 2021-04-08  │ BR      │    48 │ Male    │ Bronze       │ Referral            │
│           2 │ 2023-04-28  │ IN      │    36 │ Female  │ Silver       │ Organic             │
│           3 │ 2022-12-18  │ UK      │    35 │ Female  │ Silver       │ Organic             │
│           4 │ 2022-04-26  │ US      │    45 │ Male    │ Silver       │ Paid Search         │
│           5 │ 2022-04-20  │ IN      │    53 │ Male    │ Silver       │ Organic             │
│           6 │ 2023-07-30  │ IN      │    31 │ Female  │ Bronze       │ Paid Search         │
│           7 │ 2021-04-05  │ IN      │    35 │ Fe

In [ ]:
con.sql("SELECT * FROM raw.campaigns;")

┌─────────────┬─────────────┬──────────────┬────────────┬────────────┬────────────────┬─────────────────┐
│ campaign_id │   channel   │  objective   │ start_date │  end_date  │ target_segment │ expected_uplift │
│    int64    │   varchar   │   varchar    │    date    │    date    │    varchar     │     double      │
├─────────────┼─────────────┼──────────────┼────────────┼────────────┼────────────────┼─────────────────┤
│           1 │ Paid Search │ Cross-sell   │ 2021-10-25 │ 2021-11-26 │ Deal Seekers   │           0.022 │
│           2 │ Email       │ Retention    │ 2021-10-24 │ 2021-12-24 │ Deal Seekers   │           0.116 │
│           3 │ Email       │ Reactivation │ 2023-10-08 │ 2023-11-30 │ Churn Risk     │             0.1 │
│           4 │ Display     │ Reactivation │ 2022-07-25 │ 2022-10-07 │ Deal Seekers   │           0.111 │
│           5 │ Social      │ Acquisition  │ 2022-07-09 │ 2022-09-29 │ New Customers  │           0.144 │
│           6 │ Affiliate   │ Reactivation │ 2

In [ ]:
con.sql("SELECT * FROM raw.products;")

┌────────────┬─────────────┬───────────┬────────────┬─────────────┬────────────┐
│ product_id │  category   │   brand   │ base_price │ launch_date │ is_premium │
│   int64    │   varchar   │  varchar  │   double   │    date     │   int64    │
├────────────┼─────────────┼───────────┼────────────┼─────────────┼────────────┤
│          1 │ Grocery     │ Brand_58  │      14.19 │ 2021-08-02  │          0 │
│          2 │ Fashion     │ Brand_1   │       25.8 │ 2021-09-14  │          0 │
│          3 │ Electronics │ Brand_70  │     165.46 │ 2021-01-18  │          1 │
│          4 │ Fashion     │ Brand_56  │      75.45 │ 2023-03-03  │          1 │
│          5 │ Sports      │ Brand_1   │       72.5 │ 2022-04-19  │          1 │
│          6 │ Fashion     │ Brand_39  │      57.08 │ 2023-09-16  │          0 │
│          7 │ Beauty      │ Brand_42  │      27.23 │ 2023-05-17  │          0 │
│          8 │ Home        │ Brand_41  │     121.82 │ 2021-02-16  │          1 │
│          9 │ Home        │

In [ ]:
con.sql("SELECT * FROM raw.transactions;")

┌────────────────┬─────────────────────┬─────────────┬────────────┬──────────┬──────────────────┬───────────────┬─────────────┬─────────────┐
│ transaction_id │      timestamp      │ customer_id │ product_id │ quantity │ discount_applied │ gross_revenue │ campaign_id │ refund_flag │
│     int64      │      timestamp      │    int64    │   double   │  int64   │      double      │    double     │    int64    │    int64    │
├────────────────┼─────────────────────┼─────────────┼────────────┼──────────┼──────────────────┼───────────────┼─────────────┼─────────────┤
│              1 │ 2021-12-27 08:25:15 │       59540 │     1630.0 │        3 │              0.0 │         43.74 │           0 │           0 │
│              2 │ 2023-06-06 21:14:26 │       54871 │     1901.0 │        3 │              0.0 │        174.78 │          21 │           0 │
│              3 │ 2023-08-31 05:29:54 │       51818 │     1884.0 │        1 │              0.0 │         40.61 │          37 │           0 │
│     

In [ ]:
con.close()